In [1]:
!pip install ultralytics roboflow albumentations opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [5]:
import os
import shutil
import glob
import random
import yaml
import cv2
import albumentations as A
from google.colab import files
from tqdm import tqdm

#НАЛАШТУВАННЯ ТА ЗАВАНТАЖЕННЯ

print("Оновлення бібліотек...")
!pip install -q --upgrade kaggle

if os.path.exists("mkr-datasett.zip"):
    os.remove("mkr-datasett.zip")

print("\nПотрібен ключ API (файл kaggle.json). Завантажте його зараз:")
key_file = files.upload()

filename = list(key_file.keys())[0]
if filename != 'kaggle.json':
    os.rename(filename, 'kaggle.json')

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\nРозпочинаю скачування набору даних vashchillya/mkr-datasett...")
!kaggle datasets download -d vashchillya/mkr-datasett --force

working_folder = "/content/mkr_project_data"

if os.path.exists(working_folder):
    shutil.rmtree(working_folder)
os.makedirs(working_folder, exist_ok=True)

print("Розпаковка архіву...")
!unzip -q mkr-datasett.zip -d {working_folder}
print("Файли розпаковано.")

#НАЛАШТУВАННЯ YAML

config_matches = glob.glob(f"{working_folder}/**/data.yaml", recursive=True)

if config_matches:
    current_config = config_matches[0]
    root_loc = os.path.dirname(current_config)

    with open(current_config, 'r') as file_stream:
        yolo_settings = yaml.safe_load(file_stream)

    yolo_settings['path'] = root_loc
    yolo_settings['train'] = "train/images"
    yolo_settings['val'] = "valid/images"

    if 'test' in yolo_settings:
        yolo_settings['test'] = "test/images"

    with open(current_config, 'w') as file_stream:
        yaml.dump(yolo_settings, file_stream)

    print("Конфігурацію data.yaml успішно адаптовано!")
else:
    print("УВАГА: Файл конфігурації data.yaml відсутній! Подальша робота може бути з помилками.")
    root_loc = working_folder # Fallback

#МАСОВА АУГМЕНТАЦІЯ (РОЗМНОЖЕННЯ ДАНИХ)

train_img_dir = os.path.join(root_loc, "train", "images")
train_lbl_dir = os.path.join(root_loc, "train", "labels")

print(f"\nРозпочинаю генерацію нових даних у папці: {train_img_dir}")

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.GaussianBlur(p=0.2),
    A.RGBShift(p=0.2),
    A.RandomFog(p=0.1),
    A.CLAHE(p=0.5),
    A.ChannelShuffle(p=0.1)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

image_paths = glob.glob(os.path.join(train_img_dir, "*.jpg")) + \
              glob.glob(os.path.join(train_img_dir, "*.png")) + \
              glob.glob(os.path.join(train_img_dir, "*.jpeg"))

count_new = 0

for img_path in tqdm(image_paths, desc="Обробка зображень"):
    image = cv2.imread(img_path)
    if image is None: continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    basename = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(train_lbl_dir, basename + ".txt")

    bboxes = []
    class_labels = []

    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls = int(parts[0])
                    coords = [float(x) for x in parts[1:]] # x, y, w, h
                    bboxes.append(coords)
                    class_labels.append(cls)

    try:
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = transformed['image']
        aug_bboxes = transformed['bboxes']
        aug_labels = transformed['class_labels']

        if len(aug_bboxes) > 0:
            new_filename = basename + "_aug.jpg"
            new_txtname = basename + "_aug.txt"

            save_img_path = os.path.join(train_img_dir, new_filename)
            save_txt_path = os.path.join(train_lbl_dir, new_txtname)

            cv2.imwrite(save_img_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))

            with open(save_txt_path, 'w') as f:
                for cls, bbox in zip(aug_labels, aug_bboxes):
                    bbox = [max(0.0, min(1.0, x)) for x in bbox]
                    f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

            count_new += 1

    except Exception as e:
        pass

print(f"\nГотово! Успішно створено {count_new} нових зображень для навчання.")

Оновлення бібліотек...

Потрібен ключ API (файл kaggle.json). Завантажте його зараз:


Saving kaggle.json to kaggle (1).json

Розпочинаю скачування набору даних vashchillya/mkr-datasett...
Dataset URL: https://www.kaggle.com/datasets/vashchillya/mkr-datasett
License(s): other
 82% 6.00M/7.33M [00:00<00:00, 62.6MB/s]
100% 7.33M/7.33M [00:00<00:00, 65.7MB/s]
Розпаковка архіву...
Файли розпаковано.
Конфігурацію data.yaml успішно адаптовано!

Розпочинаю генерацію нових даних у папці: /content/mkr_project_data/train/images


Обробка зображень: 100%|██████████| 28/28 [00:05<00:00,  5.34it/s]


Готово! Успішно створено 28 нових зображень для навчання.


In [3]:
from ultralytics import YOLO

yolo_net = YOLO('yolo11n.pt')

config_path = '/content/mkr_project_data/data.yaml'

print(f"Розпочинаю навчання на конфігурації: {config_path}")

history = yolo_net.train(
    data=config_path,
    epochs=100,
    patience=0,
    batch=8,
    imgsz=640,
    name='mkr_new_run',
    optimizer='AdamW',
    lr0=0.001,
    exist_ok=True,
    verbose=True
)

print("Тренування завершено успішно!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Розпочинаю навчання на конфігурації: /content/mkr_project_data/data.yaml
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/mkr_project_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=Fa

In [4]:
import os
import glob
import cv2
from google.colab import files
from ultralytics import YOLO
from tqdm import tqdm

clip_name = "W.mp4"

if not os.path.exists(clip_name):
    print(f"Файл '{clip_name}' не знайдено. Будь ласка, завантажте відео:")
    uploaded = files.upload()

    if uploaded:
        clip_name = list(uploaded.keys())[0]
    else:
        print("Завантаження скасовано.")
        clip_name = None

if clip_name and os.path.exists(clip_name):
    print(f"Працюємо з файлом: {clip_name}")

    found_weights = glob.glob("/content/runs/detect/*/weights/best.pt")

    if found_weights:
        path_to_best = sorted(found_weights)[-1]
        model = YOLO(path_to_best)

        cap = cv2.VideoCapture(clip_name)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()

        print(f"Модель: {os.path.basename(path_to_best)}")
        print(f"Всього кадрів: {total_frames}. Починаю...")

        process_gen = model.predict(
            source=clip_name,
            save=True,
            conf=0.69,
            imgsz=640,
            stream=True,
            verbose=False
        )

        for _ in tqdm(process_gen, total=total_frames, unit="frame", desc="Обробка"):
            pass

        results_folders = sorted(glob.glob('/content/runs/detect/predict*'))
        if results_folders:
            final_path = results_folders[-1]
            print(f"\nГотово! Результат у папці: {final_path}")
            print(f"Файли: {os.listdir(final_path)}")

    else:
        print("\nПОМИЛКА: Не знайдено файл best.pt (модель не навчена).")
else:
    if clip_name:
         print(f"Файл {clip_name} відсутній.")

Файл 'W.mp4' не знайдено. Будь ласка, завантажте відео:


Saving W.mp4 to W.mp4
Працюємо з файлом: W.mp4
Модель: best.pt
Всього кадрів: 1797. Починаю...


Обробка: 100%|█████████▉| 1796/1797 [01:13<00:00, 28.02frame/s]

Results saved to /content/runs/detect/predict


Обробка: 100%|██████████| 1797/1797 [01:13<00:00, 24.45frame/s]


Готово! Результат у папці: /content/runs/detect/predict
Файли: ['W.avi']
